In [8]:
import os
import geopandas as gpd

In [9]:
tif_dir = "data/tif"

In [10]:
names = [name for name in os.listdir(tif_dir) if name.endswith(".tiff")]

In [11]:
for name in names:
    

IndentationError: expected an indented block (3552962699.py, line 2)

In [12]:
!exiftool

/bin/bash: line 1: exiftool: command not found
